In [91]:
import pandas as pd
infile = 'E:/글로벌백업(180329)/04.수수료작업/202204수수료/202204_글래드피스토스2팀 수금정산.xlsx'
outfile = 'E:/글로벌백업(180329)/04.수수료작업/202204수수료/202204_글래드피스토스2팀 수금정산_요약.xlsx'

dict_s = {'AIG손해보험':'수금','DB손보':'수금관리','MG손보':'수금','흥국화재':'장기고객관리',
   '현대해상':'수금','한화손보':'장기고객관리','KB손보':'수금','롯데손보':'수금',
   '메리츠화재':'장기수금','농협손보':'수금','삼성화재':'장기수금'}
cols =['보험사','증권번호','사원명','계약일자','계약자명','상품군','납입회차','보험료','영수일자']

df = pd.read_excel(infile,sheet_name=None,dtype={'증권번호':object})
df_all = pd.DataFrame()

for k,v in df.items():
    if k in dict_s.keys():
        temp_col = dict_s.get(k)
        df_temp = v[ cols + [temp_col] ].copy()
        df_temp.rename(columns={temp_col:'수금수수료'}, inplace=True)
        df_all = pd.concat([df_all, df_temp])

df_sum = pd.pivot_table(df_all,index=['보험사'],columns='사원명',values='수금수수료',aggfunc='sum',fill_value=0, 
               margins=True, margins_name='Total').reset_index()

with pd.ExcelWriter(outfile) as writer:
    df_sum.to_excel(writer, sheet_name='요약', index=False)
    df_all.sort_values(['사원명','보험사']).to_excel(writer, sheet_name="명세", index =False)
    workbook = writer.book
    format1 = workbook.add_format({'num_format': '#,##0'})
    worksheet = writer.sheets["요약"]
    worksheet.set_column('B:D', None, format1)
